In [4]:
# ========================================
# CAMPUSMINT - SETUP & INSTALLATION
# Run this cell first
# ========================================

!pip install -q py-algorand-sdk python-dotenv rich

print("✅ All dependencies installed!")
print("\nInstalled packages:")
print("  - py-algorand-sdk (Algorand blockchain)")
print("  - python-dotenv (environment variables)")
print("  - rich (beautiful terminal UI)")

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.2/116.2 kB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 49.0 MB/s eta 0:00:00
✅ All dependencies installed!

Installed packages:
  - py-algorand-sdk (Algorand blockchain)
  - python-dotenv (environment variables)
  - rich (beautiful terminal UI)


In [5]:
# ========================================
# CAMPUSMINT - CONFIGURATION
# Fill in your contract addresses here
# ========================================

import os
from google.colab import userdata

# ========================================
# CONTRACT ADDRESSES
# Fill these in from Person 1, 2, 3
# ========================================

CONFIG = {
    # Campus Token (CINR)
    'ASSET_ID': 755378709,  # Person 2's token

    # Person 1: Student Vault Contract
    'VAULT_APP_ID': None,  # <-- PERSON 1 FILLS THIS
    'VAULT_OWNER_ADDRESS': '',  # <-- PERSON 1 FILLS THIS

    # Person 3: NGO Treasury Contract
    'TREASURY_APP_ID': '755379222',  # <-- PERSON 3 FILLS THIS
    'TREASURY_ADDRESS': '755379212',  # <-- PERSON 3 FILLS THIS

    # Person 2: NFT Tickets
    'TICKET_ASSET_IDS': [],  # <-- PERSON 2 FILLS THIS (list of asset IDs)
    'TICKET_OWNER_ADDRESS': '',  # <-- PERSON 2 FILLS THIS

    # Algorand Network
    'ALGOD_ADDRESS': 'https://testnet-api.algonode.cloud',
    'ALGOD_TOKEN': '',
    'NETWORK': 'testnet',
}

# ========================================
# SAVE CONFIG
# ========================================

import json

with open('config.json', 'w') as f:
    json.dump(CONFIG, f, indent=2)

print("✅ Configuration saved to config.json")
print("\nCurrent config:")
print(json.dumps(CONFIG, indent=2))

# ========================================
# HELPER: Load config in other notebooks
# ========================================

def load_config():
    """Load configuration from file"""
    with open('config.json', 'r') as f:
        return json.load(f)

print("\n📝 To use in other notebooks:")
print("from config import load_config")
print("CONFIG = load_config()")

✅ Configuration saved to config.json

Current config:
{
  "ASSET_ID": 755378709,
  "VAULT_APP_ID": null,
  "VAULT_OWNER_ADDRESS": "",
  "TREASURY_APP_ID": "755379222",
  "TREASURY_ADDRESS": "755379212",
  "TICKET_ASSET_IDS": [],
  "TICKET_OWNER_ADDRESS": "",
  "ALGOD_ADDRESS": "https://testnet-api.algonode.cloud",
  "ALGOD_TOKEN": "",
  "NETWORK": "testnet"
}

📝 To use in other notebooks:
from config import load_config
CONFIG = load_config()


In [6]:
CONFIG = {
    # Multi-token setup
    'TOKENS': {
        'person_1_vault': {
            'asset_id': None,  # Person 1 fills
            'name': 'Vault Token',
            'peg': '1:1 INR'
        },
        'person_2_tickets': {
            'asset_id': 755378709,
            'name': 'Campus INR',
            'peg': '1:1 INR'
        },
        'person_3_treasury': {
            'asset_id': 755379212,
            'name': 'CampusCoin',
            'peg': '1:1 INR'
        }
    },

    # Contract mapping
    'CONTRACTS': {
        'vault': {
            'app_id': None,  # Person 1 fills
            'token_used': 'person_1_vault'
        },
        'treasury': {
            'app_id': 755379222,
            'token_used': 'person_3_treasury'
        },
        'tickets': {
            'asset_ids': [],  # Person 2 fills
            'payment_token': 'person_2_tickets'
        }
    },

    # Exchange logic (backend handles)
    'EXCHANGE_ENABLED': True,
    'EXCHANGE_RATE': 1.0  # All tokens = 1 INR
}

In [7]:
class CampusMintCoordinator:
    """
    Coordinates between independent contracts using different tokens.
    Handles token equivalence and cross-contract workflows.
    """

    def __init__(self, config):
        self.tokens = config['TOKENS']
        self.contracts = config['CONTRACTS']
        self.exchange_rate = config['EXCHANGE_RATE']

    def verify_payment(self, contract_name, amount_inr):
        """
        Verify payment in ANY token (all pegged 1:1 to INR)
        """
        contract = self.contracts[contract_name]
        token_key = contract['token_used']
        token_asset_id = self.tokens[token_key]['asset_id']

        # Read from blockchain
        payment_verified = self.check_blockchain_payment(
            token_asset_id,
            amount_inr
        )

        return payment_verified

    def coordinate_ticket_purchase(self, student_wallet, amount):
        """
        Coordinate: Treasury payment → Ticket NFT release
        """
        # 1. Verify treasury contribution
        treasury_token = self.tokens['person_3_treasury']['asset_id']
        contribution_verified = self.verify_contribution(
            treasury_token,
            student_wallet,
            amount
        )

        if contribution_verified:
            # 2. Trigger ticket NFT transfer
            ticket_nft = self.get_available_ticket()
            self.transfer_ticket(ticket_nft, student_wallet)

            return {
                'success': True,
                'paid_with': treasury_token,
                'received': ticket_nft
            }

    def get_unified_balance(self, wallet_address):
        """
        Show student's total balance across ALL tokens in INR
        """
        balances = {}
        total_inr = 0

        for token_key, token_info in self.tokens.items():
            asset_id = token_info['asset_id']
            if asset_id:
                balance = self.get_token_balance(wallet_address, asset_id)
                balances[token_key] = balance
                total_inr += balance  # 1:1 peg

        return {
            'individual': balances,
            'total_inr': total_inr
        }

In [8]:
def show_multi_token_panel(self):
    """Display all tokens with equivalence"""

    table = Table(
        title="🏦 Campus Tokens (All 1:1 INR Pegged)",
        box=box.ROUNDED
    )
    table.add_column("Token", style="cyan")
    table.add_column("Asset ID", style="yellow")
    table.add_column("Used By", style="green")
    table.add_column("Status", style="blue")

    table.add_row(
        "Vault Token",
        str(CONFIG['TOKENS']['person_1_vault']['asset_id']),
        "Student Vault",
        "✅ Active"
    )

    table.add_row(
        "Campus INR",
        "755378709",
        "Event Tickets",
        "✅ Active"
    )

    table.add_row(
        "CampusCoin",
        "755379212",
        "Club Treasury",
        "✅ Active"
    )

    console.print(table)
    console.print("\n[dim]💱 Exchange Rate: 1 Token = ₹1 INR (all tokens)[/dim]")

In [12]:
# ============================================================================
# CAMPUSMINT - MULTI-TOKEN DASHBOARD
# Complete Colab Notebook - Ready to Run
# ============================================================================

# CELL 1: INSTALL DEPENDENCIES
# ============================================================================

!pip install -q py-algorand-sdk rich tabulate

print("✅ Dependencies installed!")

# ============================================================================
# CELL 2: IMPORTS & CONFIGURATION
# ============================================================================

from algosdk.v2client import algod
from algosdk import account, mnemonic
import json
import base64
from datetime import datetime
import time

from rich.console import Console
from rich.panel import Panel
from rich.table import Table
from rich.progress import Progress, BarColumn, TextColumn
from rich import box
from rich.layout import Layout
from rich.live import Live

console = Console()

# ============================================================================
# CONFIGURATION - FILL IN YOUR VALUES
# ============================================================================

CONFIG = {
    # Network
    'ALGOD_ADDRESS': 'https://testnet-api.algonode.cloud',
    'ALGOD_TOKEN': '',
    'NETWORK': 'testnet',

    # Person 1: Student Vault
    'VAULT': {
        'app_id': None,  # <-- PERSON 1 FILLS THIS
        'token_asset_id': None,  # <-- PERSON 1 FILLS THIS
        'token_name': 'Vault Token',
        'owner_address': '',
        'status': 'pending'  # pending/deployed
    },

    # Person 2: NFT Tickets
    'TICKETS': {
        'payment_token_id': 755378709,  # Campus INR
        'token_name': 'Campus INR',
        'ticket_asset_ids': [],  # <-- PERSON 2 FILLS THIS (list)
        'owner_address': '',
        'status': 'deployed'
    },

    # Person 3: NGO Treasury
    'TREASURY': {
        'app_id': 755379222,
        'token_asset_id': 755379212,  # CampusCoin
        'token_name': 'CampusCoin',
        'app_address': 'VFRXZACPTSFFYPFPMKKBSRZVYB7B5JYNE4Q4BQYRQYREQANPFZ7YXN3BI',
        'owner_address': 'C57VRWFTIRL567HG7BFCY3DEC2QLN6TDZVPFD3AG7NZDOD2JVVYQJVCA4',
        'status': 'deployed'
    },

    # Token Exchange (All pegged 1:1 to INR)
    'EXCHANGE_RATE': 1.0,
    'BASE_CURRENCY': 'INR'
}

# Save config
with open('campusmint_config.json', 'w') as f:
    json.dump(CONFIG, f, indent=2)

print("✅ Configuration loaded!")
print(json.dumps(CONFIG, indent=2))

# ============================================================================
# CELL 3: BLOCKCHAIN CLIENT SETUP
# ============================================================================

# Initialize Algorand client
algod_client = algod.AlgodClient(
    CONFIG['ALGOD_TOKEN'],
    CONFIG['ALGOD_ADDRESS']
)

# Test connection
try:
    status = algod_client.status()
    console.print(f"[green]✅ Connected to Algorand {CONFIG['NETWORK']}[/green]")
    console.print(f"[dim]Last Round: {status['last-round']}[/dim]")
except Exception as e:
    console.print(f"[red]❌ Connection failed: {e}[/red]")

# ============================================================================
# CELL 4: BLOCKCHAIN READER FUNCTIONS
# ============================================================================

def get_asset_info(asset_id):
    """Get token information from blockchain"""
    try:
        asset_info = algod_client.asset_info(asset_id)
        params = asset_info['params']

        return {
            'asset_id': asset_id,
            'name': params.get('name', 'Unknown'),
            'unit_name': params.get('unit-name', 'Unknown'),
            'total': params.get('total', 0) / (10 ** params.get('decimals', 0)),
            'decimals': params.get('decimals', 0),
            'creator': params.get('creator', 'Unknown'),
            'url': params.get('url', ''),
            'manager': params.get('manager', ''),
        }
    except Exception as e:
        console.print(f"[red]Error reading asset {asset_id}: {e}[/red]")
        return None


def get_account_balance(address, asset_id=None):
    """Get ALGO or token balance for an address"""
    try:
        account_info = algod_client.account_info(address)

        if asset_id:
            # Get specific token balance
            assets = account_info.get('assets', [])
            for asset in assets:
                if asset['asset-id'] == asset_id:
                    # Assuming 2 decimals for all tokens
                    return asset['amount'] / 100
            return 0  # Not opted in
        else:
            # Get ALGO balance
            return account_info.get('amount', 0) / 1_000_000

    except Exception as e:
        console.print(f"[red]Error reading balance: {e}[/red]")
        return 0


def read_app_global_state(app_id):
    """Read application global state"""
    if not app_id:
        return {}

    try:
        app_info = algod_client.application_info(app_id)
        global_state = app_info['params'].get('global-state', [])

        state = {}
        for item in global_state:
            # Decode key
            key = base64.b64decode(item['key']).decode('utf-8')

            # Decode value
            if item['value']['type'] == 1:  # bytes
                try:
                    value = base64.b64decode(item['value']['bytes']).decode('utf-8')
                except:
                    value = base64.b64decode(item['value']['bytes']).hex()
            else:  # uint
                value = item['value']['uint']

            state[key] = value

        return state

    except Exception as e:
        console.print(f"[red]Error reading app {app_id}: {e}[/red]")
        return {}


def read_vault_state(app_id):
    """Parse vault contract state"""
    if not app_id:
        return {
            'status': 'Not deployed',
            'locked_amount': 0,
            'unlock_time': 'N/A',
            'is_unlocked': False
        }

    try:
        state = read_app_global_state(app_id)

        locked_amount = state.get('locked_amount', 0) / 100
        unlock_time = state.get('unlock_time', 0)

        if unlock_time:
            unlock_date = datetime.fromtimestamp(unlock_time).strftime('%Y-%m-%d %H:%M')
            current_time = int(datetime.now().timestamp())
            is_unlocked = current_time >= unlock_time
        else:
            unlock_date = 'N/A'
            is_unlocked = False

        return {
            'status': 'Active',
            'owner': state.get('owner', 'Unknown'),
            'beneficiary': state.get('beneficiary', 'Unknown'),
            'locked_amount': locked_amount,
            'unlock_time': unlock_date,
            'unlock_timestamp': unlock_time,
            'is_unlocked': is_unlocked,
            'purpose': state.get('purpose', 'N/A'),
            'raw_state': state
        }

    except Exception as e:
        return {
            'status': 'Error',
            'locked_amount': 0,
            'unlock_time': 'N/A',
            'is_unlocked': False,
            'error': str(e)
        }


def read_treasury_state(app_id):
    """Parse treasury contract state"""
    if not app_id:
        return {
            'status': 'Not deployed',
            'goal': 0,
            'raised': 0,
            'progress': 0
        }

    try:
        state = read_app_global_state(app_id)

        goal = state.get('goal', state.get('GOAL', 0)) / 100
        total_raised = state.get('total_raised', state.get('total', state.get('TOTAL', 0))) / 100
        deadline = state.get('deadline', state.get('DEADLINE', 0))

        if deadline:
            deadline_date = datetime.fromtimestamp(deadline).strftime('%Y-%m-%d %H:%M')
            current_time = int(datetime.now().timestamp())
            is_active = current_time < deadline
        else:
            deadline_date = 'N/A'
            is_active = True

        progress = (total_raised / goal * 100) if goal > 0 else 0

        return {
            'status': 'Active' if is_active else 'Ended',
            'admin': state.get('admin', state.get('creator', state.get('CREATOR', 'Unknown'))),
            'goal': goal,
            'raised': total_raised,
            'deadline': deadline_date,
            'deadline_timestamp': deadline,
            'progress': progress,
            'is_active': is_active,
            'beneficiary': state.get('beneficiary', 'Unknown'),
            'asset_id': state.get('asset_id', state.get('ASSET', 'Unknown')),
            'raw_state': state
        }

    except Exception as e:
        return {
            'status': 'Error',
            'goal': 0,
            'raised': 0,
            'progress': 0,
            'error': str(e)
        }


def read_ticket_info(asset_id):
    """Read NFT ticket information"""
    try:
        info = get_asset_info(asset_id)
        if info:
            info['is_nft'] = info['total'] == 1
            return info
        return None
    except Exception as e:
        console.print(f"[red]Error reading ticket {asset_id}: {e}[/red]")
        return None

print("✅ Blockchain reader functions loaded!")

# ============================================================================
# CELL 5: DASHBOARD CLASS
# ============================================================================

class CampusMintDashboard:

    def __init__(self, config):
        self.config = config
        self.console = Console()

    def clear(self):
        """Clear console"""
        self.console.clear()

    # ========================================================================
    # HEADER
    # ========================================================================

    def show_header(self):
        """Display dashboard header"""
        header = """
╔═══════════════════════════════════════════════════════════════╗
║                                                               ║
║             🏦  CAMPUSMINT CONTROL PANEL  🏦                  ║
║                                                               ║
║        Decentralized Campus Finance on Algorand               ║
║              Multi-Token Architecture (v0.6)                  ║
║                                                               ║
╚═══════════════════════════════════════════════════════════════╝
        """
        self.console.print(Panel(header, style="bold cyan", border_style="cyan"))

    # ========================================================================
    # TOKEN REGISTRY
    # ========================================================================

    def show_token_registry(self):
        """Display all tokens with 1:1 INR peg"""
        table = Table(
            title="🪙 Multi-Token Registry (All 1:1 INR Pegged)",
            box=box.ROUNDED,
            title_style="bold green"
        )
        table.add_column("Token", style="cyan", width=15)
        table.add_column("Asset ID", style="yellow", width=12)
        table.add_column("Used By", style="green", width=18)
        table.add_column("Supply", style="magenta", width=15)
        table.add_column("Status", style="blue", width=10)

        # Person 1: Vault Token
        vault_token_id = self.config['VAULT']['token_asset_id']
        if vault_token_id:
            try:
                vault_token = get_asset_info(vault_token_id)
                table.add_row(
                    vault_token['name'][:15],
                    str(vault_token_id),
                    "Student Vault",
                    f"₹{vault_token['total']:,.0f}",
                    "✅ Active"
                )
            except:
                table.add_row(
                    "Vault Token",
                    str(vault_token_id),
                    "Student Vault",
                    "N/A",
                    "⚠️ Error"
                )
        else:
            table.add_row(
                "Vault Token",
                "Not set",
                "Student Vault",
                "N/A",
                "⏳ Pending"
            )

        # Person 2: Tickets Token
        ticket_token_id = self.config['TICKETS']['payment_token_id']
        try:
            ticket_token = get_asset_info(ticket_token_id)
            table.add_row(
                ticket_token['name'][:15],
                str(ticket_token_id),
                "Event Tickets",
                f"₹{ticket_token['total']:,.0f}",
                "✅ Active"
            )
        except:
            table.add_row(
                "Campus INR",
                str(ticket_token_id),
                "Event Tickets",
                "N/A",
                "⚠️ Error"
            )

        # Person 3: Treasury Token
        treasury_token_id = self.config['TREASURY']['token_asset_id']
        try:
            treasury_token = get_asset_info(treasury_token_id)
            table.add_row(
                treasury_token['name'][:15],
                str(treasury_token_id),
                "Club Treasury",
                f"₹{treasury_token['total']:,.0f}",
                "✅ Active"
            )
        except:
            table.add_row(
                "CampusCoin",
                str(treasury_token_id),
                "Club Treasury",
                "N/A",
                "⚠️ Error"
            )

        self.console.print(table)
        self.console.print("\n[dim]💱 Exchange Rate: 1 Token = ₹1 INR (all tokens equivalent)[/dim]")

    # ========================================================================
    # VAULT PANEL
    # ========================================================================

    def show_vault_panel(self):
        """Display Student Vault status"""
        app_id = self.config['VAULT']['app_id']

        table = Table(
            title="🔐 Student Vault Contract (Person 1)",
            box=box.ROUNDED,
            show_header=False,
            title_style="bold yellow"
        )
        table.add_column("Property", style="cyan", width=20)
        table.add_column("Value", style="yellow")

        if not app_id:
            table.add_row("Status", "⏳ Not deployed yet")
            table.add_row("Owner", self.config['VAULT']['owner_address'] or "Not set")
            table.add_row("Token", self.config['VAULT']['token_name'])
            self.console.print(table)
            return

        try:
            vault_state = read_vault_state(app_id)

            table.add_row("App ID", str(app_id))
            table.add_row("Status", vault_state['status'])
            table.add_row("Locked Amount", f"₹{vault_state['locked_amount']:,.2f}")
            table.add_row("Unlock Time", vault_state['unlock_time'])
            table.add_row("Lock Status", "✅ Unlocked" if vault_state['is_unlocked'] else "🔒 Locked")
            table.add_row("Purpose", str(vault_state.get('purpose', 'N/A')))
            table.add_row("Token Used", self.config['VAULT']['token_name'])

            self.console.print(table)

        except Exception as e:
            table.add_row("Status", f"❌ Error: {str(e)[:50]}")
            self.console.print(table)

    # ========================================================================
    # TREASURY PANEL
    # ========================================================================

    def show_treasury_panel(self):
        """Display Treasury Pool status"""
        app_id = self.config['TREASURY']['app_id']

        table = Table(
            title="💰 NGO Treasury Pool Contract (Person 3)",
            box=box.ROUNDED,
            show_header=False,
            title_style="bold magenta"
        )
        table.add_column("Property", style="cyan", width=20)
        table.add_column("Value", style="magenta")

        if not app_id:
            table.add_row("Status", "⏳ Not deployed")
            self.console.print(table)
            return

        try:
            treasury_state = read_treasury_state(app_id)

            table.add_row("App ID", str(app_id))
            table.add_row("Status", treasury_state['status'])
            table.add_row("Goal", f"₹{treasury_state['goal']:,.2f}")
            table.add_row("Raised", f"₹{treasury_state['raised']:,.2f}")
            table.add_row("Progress", f"{treasury_state['progress']:.1f}%")
            table.add_row("Deadline", treasury_state['deadline'])
            table.add_row("Token Used", self.config['TREASURY']['token_name'])

            self.console.print(table)

            # Progress bar
            if treasury_state['goal'] > 0:
                self.console.print()
                with Progress() as progress:
                    task = progress.add_task(
                        "[magenta]Funding Progress",
                        total=100
                    )
                    progress.update(task, completed=treasury_state['progress'])
                    time.sleep(0.5)  # Show progress bar briefly

        except Exception as e:
            table.add_row("Status", f"❌ Error: {str(e)[:50]}")
            self.console.print(table)

    # ========================================================================
    # TICKETS PANEL
    # ========================================================================

    def show_tickets_panel(self):
        """Display NFT Tickets"""
        ticket_ids = self.config['TICKETS']['ticket_asset_ids']

        table = Table(
            title="🎟️ Event Tickets (NFTs) - Person 2",
            box=box.ROUNDED,
            title_style="bold blue"
        )
        table.add_column("Asset ID", style="cyan", width=12)
        table.add_column("Ticket Name", style="green", width=30)
        table.add_column("Type", style="yellow", width=10)
        table.add_column("Status", style="blue", width=10)

        if not ticket_ids:
            table.add_row("N/A", "⏳ No tickets minted yet", "N/A", "Pending")
            self.console.print(table)
            self.console.print(f"\n[dim]Payment Token: {self.config['TICKETS']['token_name']} (Asset {self.config['TICKETS']['payment_token_id']})[/dim]")
            return

        try:
            for asset_id in ticket_ids[:10]:  # Show first 10
                ticket_info = read_ticket_info(asset_id)
                if ticket_info:
                    table.add_row(
                        str(asset_id),
                        ticket_info['name'][:30],
                        "NFT" if ticket_info['is_nft'] else "Token",
                        "✅ Minted"
                    )
                else:
                    table.add_row(
                        str(asset_id),
                        "Error reading",
                        "?",
                        "❌ Error"
                    )

            if len(ticket_ids) > 10:
                table.add_row("...", f"+ {len(ticket_ids) - 10} more tickets", "...", "...")

            self.console.print(table)
            self.console.print(f"\n[dim]Total Tickets: {len(ticket_ids)} | Payment Token: {self.config['TICKETS']['token_name']}[/dim]")

        except Exception as e:
            self.console.print(f"[red]Error displaying tickets: {e}[/red]")

    # ========================================================================
    # MENU
    # ========================================================================

    def show_menu(self):
        """Display main menu"""
        menu_table = Table(
            title="📋 Available Actions",
            box=box.SIMPLE,
            show_header=False,
            title_style="bold white"
        )
        menu_table.add_column("Option", style="cyan", width=5)
        menu_table.add_column("Action", style="white")

        menu_table.add_row("1", "🔄 Refresh Dashboard")
        menu_table.add_row("2", "🔗 View AlgoExplorer Links")
        menu_table.add_row("3", "📊 Run Demo Scenario")
        menu_table.add_row("4", "📝 Export State Report")
        menu_table.add_row("5", "⚙️  Show Raw Configuration")
        menu_table.add_row("6", "📈 Show Analytics")
        menu_table.add_row("0", "❌ Exit")

        self.console.print(menu_table)

    # ========================================================================
    # ALGOEXPLORER LINKS
    # ========================================================================

    def show_algoexplorer_links(self):
        """Display all AlgoExplorer links"""
        self.console.clear()
        self.console.print("\n[bold cyan]🔗 AlgoExplorer Links (Testnet)[/bold cyan]\n")

        links_table = Table(box=box.ROUNDED)
        links_table.add_column("Component", style="cyan", width=25)
        links_table.add_column("Link", style="blue")

        # Tokens
        if self.config['VAULT']['token_asset_id']:
            vault_token_link = f"https://testnet.algoexplorer.io/asset/{self.config['VAULT']['token_asset_id']}"
            links_table.add_row("Vault Token", vault_token_link)

        ticket_token_link = f"https://testnet.algoexplorer.io/asset/{self.config['TICKETS']['payment_token_id']}"
        links_table.add_row("Tickets Token (Campus INR)", ticket_token_link)

        treasury_token_link = f"https://testnet.algoexplorer.io/asset/{self.config['TREASURY']['token_asset_id']}"
        links_table.add_row("Treasury Token (CampusCoin)", treasury_token_link)

        # Contracts
        if self.config['VAULT']['app_id']:
            vault_link = f"https://testnet.algoexplorer.io/application/{self.config['VAULT']['app_id']}"
            links_table.add_row("Student Vault Contract", vault_link)

        treasury_link = f"https://testnet.algoexplorer.io/application/{self.config['TREASURY']['app_id']}"
        links_table.add_row("Treasury Pool Contract", treasury_link)

        # NFT Tickets
        if self.config['TICKETS']['ticket_asset_ids']:
            for i, asset_id in enumerate(self.config['TICKETS']['ticket_asset_ids'][:5], 1):
                ticket_link = f"https://testnet.algoexplorer.io/asset/{asset_id}"
                links_table.add_row(f"Event Ticket #{i}", ticket_link)

        self.console.print(links_table)
        self.console.print("\n[dim]💡 Copy these links to verify on blockchain explorer[/dim]\n")

    # ========================================================================
    # DEMO SCENARIO
    # ========================================================================

    def run_demo_scenario(self):
        """Run automated demo showcasing all features"""
        self.console.clear()
        self.console.print(Panel(
            "[bold cyan]🎬 CampusMint Demo Scenario[/bold cyan]\n"
            "[dim]Demonstrating multi-token campus finance system...[/dim]",
            border_style="cyan"
        ))

        time.sleep(1)

        # Scenario 1: Student Vault
        self.console.print("\n[bold yellow]━━━ Scenario 1: Student Savings Vault ━━━[/bold yellow]")
        self.console.print("\n📖 Story: Student saves ₹5,000 for exam fees using Vault Token\n")

        time.sleep(1)
        self.console.print("  1️⃣ Student creates time-locked vault")
        time.sleep(0.7)
        self.console.print("  2️⃣ Deposits ₹5,000 (Vault Token)")
        time.sleep(0.7)
        self.console.print("  3️⃣ Vault locked until May 2025")
        time.sleep(0.7)

        self.console.print("\n  ✅ [green]Vault created successfully![/green]")
        self.console.print("  🔒 Early withdrawal: [red]REJECTED[/red] (blockchain enforced)")
        self.console.print("  🪙 Token Used: Vault Token (Independent)")

        time.sleep(2)

        # Scenario 2: Treasury Pool
        self.console.print("\n[bold yellow]━━━ Scenario 2: Club Fundraising ━━━[/bold yellow]")
        self.console.print("\n📖 Story: Tech club raises ₹100,000 using CampusCoin\n")

        time.sleep(1)
        self.console.print("  1️⃣ Club creates treasury pool (Goal: ₹100,000)")
        time.sleep(0.7)
        self.console.print("  2️⃣ Students contribute using CampusCoin")
        time.sleep(0.7)
        self.console.print("  3️⃣ Contributions tracked on-chain")
        time.sleep(0.7)
        self.console.print("  4️⃣ Current: ₹50,000 raised (50%)")
        time.sleep(0.7)

        self.console.print("\n  ✅ [green]Treasury pool active![/green]")
        self.console.print("  📊 Transparent contribution tracking")
        self.console.print("  🪙 Token Used: CampusCoin (Asset 755379212)")

        time.sleep(2)

        # Scenario 3: NFT Tickets
        self.console.print("\n[bold yellow]━━━ Scenario 3: Event Ticketing ━━━[/bold yellow]")
        self.console.print("\n📖 Story: Student buys fest ticket using Campus INR\n")

        time.sleep(1)
        self.console.print("  1️⃣ Club mints unique NFT tickets")
        time.sleep(0.7)
        self.console.print("  2️⃣ Student pays ₹500 (Campus INR token)")
        time.sleep(0.7)
        self.console.print("  3️⃣ Backend coordinates payment verification")
        time.sleep(0.7)
        self.console.print("  4️⃣ Ticket NFT transferred to student")
        time.sleep(0.7)

        self.console.print("\n  ✅ [green]Ticket purchase complete![/green]")
        self.console.print("  🎟️ NFT prevents screenshot fraud")
        self.console.print("  🪙 Payment Token: Campus INR (Asset 755378709)")

        time.sleep(2)

        # Multi-Token Coordination
        self.console.print("\n[bold yellow]━━━ Backend Coordination ━━━[/bold yellow]")
        self.console.print("\n📖 How different tokens work together:\n")

        time.sleep(1)
        self.console.print("  💱 All tokens pegged 1:1 to INR")
        time.sleep(0.7)
        self.console.print("  🔄 Backend converts between tokens")
        time.sleep(0.7)
        self.console.print("  📡 Contracts remain independent")
        time.sleep(0.7)
        self.console.print("  ✅ Coordination via off-chain logic")
        time.sleep(0.7)

        # Summary
        self.console.print("\n" + "=" * 70)
        self.console.print("\n[bold green]✅ Demo Completed Successfully![/bold green]\n")

        summary_table = Table(box=box.ROUNDED, title="Demo Summary", title_style="bold green")
        summary_table.add_column("Feature", style="cyan")
        summary_table.add_column("Token Used", style="yellow")
        summary_table.add_column("Status", style="green")

        summary_table.add_row(
            "Student Vault",
            self.config['VAULT']['token_name'],
            "✅ Independent"
        )
        summary_table.add_row(
            "Treasury Pool",
            "CampusCoin (755379212)",
            "✅ Independent"
        )
        summary_table.add_row(
            "NFT Tickets",
            "Campus INR (755378709)",
            "✅ Independent"
        )
        summary_table.add_row(
            "Backend Coordination",
            "All tokens (1:1 INR)",
            "✅ Functional"
        )

        self.console.print(summary_table)

        self.console.print("\n[bold cyan]🔗 All contracts verifiable on AlgoExplorer[/bold cyan]")
        self.console.print("[dim]This demonstrates modular campus finance infrastructure.[/dim]\n")

    # ========================================================================
    # EXPORT REPORT
    # ========================================================================

    def export_state_report(self):
        """Generate and save comprehensive state report"""
        self.console.print("\n[bold cyan]📝 Generating State Report...[/bold cyan]\n")

        report = {
            'timestamp': datetime.now().isoformat(),
            'network': self.config['NETWORK'],
            'architecture': 'Multi-Token (3 independent tokens)',
            'tokens': {},
            'vault': {},
            'treasury': {},
            'tickets': []
        }

        # Token info
        for token_key in ['VAULT', 'TICKETS', 'TREASURY']:
            token_config = self.config[token_key]
            if token_key == 'TICKETS':
                asset_id = token_config['payment_token_id']
            else:
                asset_id = token_config['token_asset_id']

            if asset_id:
                try:
                    token_info = get_asset_info(asset_id)
                    report['tokens'][token_key.lower()] = token_info
                except:
                    pass

        # Vault state
        if self.config['VAULT']['app_id']:
            try:
                vault_state = read_vault_state(self.config['VAULT']['app_id'])
                report['vault'] = vault_state
            except:
                pass

        # Treasury state
        if self.config['TREASURY']['app_id']:
            try:
                treasury_state = read_treasury_state(self.config['TREASURY']['app_id'])
                report['treasury'] = treasury_state
            except:
                pass

        # Tickets
        for asset_id in self.config['TICKETS']['ticket_asset_ids']:
            try:
                ticket_info = read_ticket_info(asset_id)
                if ticket_info:
                    report['tickets'].append(ticket_info)
            except:
                pass

        # Save report
        filename = f"campusmint_report_{int(time.time())}.json"
        with open(filename, 'w') as f:
            json.dump(report, f, indent=2)

        self.console.print(f"[green]✅ Report saved: {filename}[/green]")
        self.console.print(f"[dim]Contains complete blockchain state snapshot[/dim]\n")

        # Show summary
        summary_table = Table(title="Report Summary", box=box.ROUNDED)
        summary_table.add_column("Metric", style="cyan")
        summary_table.add_column("Value", style="green")

        summary_table.add_row("Tokens Tracked", str(len(report['tokens'])))
        summary_table.add_row("Vault Status", report['vault'].get('status', 'N/A'))
        summary_table.add_row("Treasury Status", report['treasury'].get('status', 'N/A'))
        summary_table.add_row("Tickets Minted", str(len(report['tickets'])))

        self.console.print(summary_table)

    # ========================================================================
    # ANALYTICS
    # ========================================================================

    def show_analytics(self):
        """Display system analytics"""
        self.console.clear()
        self.console.print("\n[bold cyan]📈 CampusMint Analytics[/bold cyan]\n")

        analytics_table = Table(box=box.ROUNDED, title="System Metrics")
        analytics_table.add_column("Metric", style="cyan", width=30)
        analytics_table.add_column("Value", style="green", width=20)
        analytics_table.add_column("Details", style="yellow")

        # Count deployed contracts
        deployed_contracts = 0
        if self.config['VAULT']['app_id']:
            deployed_contracts += 1
        if self.config['TREASURY']['app_id']:
            deployed_contracts += 1

        analytics_table.add_row(
            "Deployed Contracts",
            str(deployed_contracts),
            "Vault + Treasury"
        )

        # Count tokens
        unique_tokens = set()
        if self.config['VAULT']['token_asset_id']:
            unique_tokens.add(self.config['VAULT']['token_asset_id'])
        unique_tokens.add(self.config['TICKETS']['payment_token_id'])
        unique_tokens.add(self.config['TREASURY']['token_asset_id'])

        analytics_table.add_row(
            "Unique Tokens",
            str(len(unique_tokens)),
            "All pegged 1:1 INR"
        )

        # Treasury metrics
        if self.config['TREASURY']['app_id']:
            treasury_state = read_treasury_state(self.config['TREASURY']['app_id'])
            analytics_table.add_row(
                "Treasury Goal",
                f"₹{treasury_state['goal']:,.0f}",
                f"{treasury_state['progress']:.1f}% achieved"
            )
            analytics_table.add_row(
                "Total Raised",
                f"₹{treasury_state['raised']:,.0f}",
                "CampusCoin token"
            )

        # Tickets
        ticket_count = len(self.config['TICKETS']['ticket_asset_ids'])
        analytics_table.add_row(
            "NFT Tickets Minted",
            str(ticket_count),
            "Campus INR payment"
        )

        # Vault
        if self.config['VAULT']['app_id']:
            vault_state = read_vault_state(self.config['VAULT']['app_id'])
            analytics_table.add_row(
                "Vault Locked Amount",
                f"₹{vault_state['locked_amount']:,.0f}",
                "Time-locked savings"
            )

        self.console.print(analytics_table)

        # Token distribution
        self.console.print("\n[bold]Token Distribution:[/bold]\n")
        dist_table = Table(box=box.SIMPLE)
        dist_table.add_column("Token", style="cyan")
        dist_table.add_column("Asset ID", style="yellow")
        dist_table.add_column("Used By", style="green")

        if self.config['VAULT']['token_asset_id']:
            dist_table.add_row(
                self.config['VAULT']['token_name'],
                str(self.config['VAULT']['token_asset_id']),
                "Student Vault"
            )

        dist_table.add_row(
            "Campus INR",
            str(self.config['TICKETS']['payment_token_id']),
            "Event Tickets"
        )

        dist_table.add_row(
            "CampusCoin",
            str(self.config['TREASURY']['token_asset_id']),
            "Club Treasury"
        )

        self.console.print(dist_table)

    # ========================================================================
    # MAIN LOOP
    # ========================================================================

    def run(self):
        """Main dashboard loop"""
        while True:
            self.clear()
            self.show_header()

            self.console.print()
            self.show_token_registry()

            self.console.print()
            self.show_vault_panel()

            self.console.print()
            self.show_treasury_panel()

            self.console.print()
            self.show_tickets_panel()

            self.console.print()
            self.show_menu()

            choice = input("\n[?] Select option (0-6): ").strip()

            if choice == "0":
                self.console.print("\n[yellow]👋 Goodbye! CampusMint Dashboard closed.[/yellow]\n")
                break

            elif choice == "1":
                continue  # Refresh

            elif choice == "2":
                self.show_algoexplorer_links()
                input("\nPress Enter to continue...")

            elif choice == "3":
                self.run_demo_scenario()
                input("\nPress Enter to continue...")

            elif choice == "4":
                self.export_state_report()
                input("\nPress Enter to continue...")

            elif choice == "5":
                self.console.print("\n[bold cyan]⚙️  Raw Configuration:[/bold cyan]\n")
                self.console.print(json.dumps(self.config, indent=2))
                input("\nPress Enter to continue...")

            elif choice == "6":
                self.show_analytics()
                input("\nPress Enter to continue...")

            else:
                self.console.print("[red]Invalid option![/red]")
                time.sleep(1)

print("✅ Dashboard class loaded!")

# ============================================================================
# CELL 6: RUN DASHBOARD
# ============================================================================

console.print("\n[bold green]🚀 Starting CampusMint Dashboard...[/bold green]\n")
time.sleep(1)

dashboard = CampusMintDashboard(CONFIG)
dashboard.run()

# ============================================================================
# END OF NOTEBOOK
# ============================================================================

✅ Dependencies installed!
✅ Configuration loaded!
{
  "ALGOD_ADDRESS": "https://testnet-api.algonode.cloud",
  "ALGOD_TOKEN": "",
  "NETWORK": "testnet",
  "VAULT": {
    "app_id": null,
    "token_asset_id": null,
    "token_name": "Vault Token",
    "owner_address": "",
    "status": "pending"
  },
  "TICKETS": {
    "payment_token_id": 755378709,
    "token_name": "Campus INR",
    "ticket_asset_ids": [],
    "owner_address": "",
    "status": "deployed"
  },
  "TREASURY": {
    "app_id": 755379222,
    "token_asset_id": 755379212,
    "token_name": "CampusCoin",
    "app_address": "VFRXZACPTSFFYPFPMKKBSRZVYB7B5JYNE4Q4BQYRQYREQANPFZ7YXN3BI",
    "owner_address": "C57VRWFTIRL567HG7BFCY3DEC2QLN6TDZVPFD3AG7NZDOD2JVVYQJVCA4",
    "status": "deployed"
  },
  "EXCHANGE_RATE": 1.0,
  "BASE_CURRENCY": "INR"
}


✅ Connected to Algorand testnet

Last Round: 60454084

✅ Blockchain reader functions loaded!
✅ Dashboard class loaded!


🚀 Starting CampusMint Dashboard...

...

                     🎟️ Event Tickets (NFTs) - Person 2                     
╭──────────────┬────────────────────────────────┬────────────┬────────────╮
│ Asset ID     │ Ticket Name                    │ Type       │ Status     │
├──────────────┼────────────────────────────────┼────────────┼────────────┤
│ N/A          │ ⏳ No tickets minted yet       │ N/A        │ Pending    │
╰──────────────┴────────────────────────────────┴────────────┴────────────╯

Payment Token: Campus INR (Asset 755378709)

         📋 Available Actions         
                                      
  1       🔄 Refresh Dashboard        
  2       🔗 View AlgoExplorer Links  
  3       📊 Run Demo Scenario        
  4       📝 Export State Report      
  5       ⚙️  Show Raw Configuration   
  6       📈 Show Analytics           
  0       ❌ Exit                    


[?] Select option (0-6): 0


👋 Goodbye! CampusMint Dashboard closed.